<a href="https://colab.research.google.com/github/pmy0792/pytorch_tutorial/blob/main/lab_10_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader

from matplotlib.pyplot import imshow
%matplotlib inline

In [14]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)

In [15]:
trans = transforms.Compose([
    transforms.Resize((64,128))
])

train_data = torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/Colab Notebooks/custom_data/origin_data', transform=trans)

In [17]:
for num, value in enumerate(train_data):
    data, label = value
    print(num, data, label)
    
    if(label == 0):
        data.save('/content/drive/MyDrive/Colab Notebooks/custom_data/train_data/gray/%d_%d.jpeg'%(num, label))
    else:
        data.save('/content/drive/MyDrive/Colab Notebooks/custom_data/train_data/red/%d_%d.jpeg'%(num, label))

0 <PIL.Image.Image image mode=RGB size=128x64 at 0x7F89B5016E10> 0
1 <PIL.Image.Image image mode=RGB size=128x64 at 0x7F89B5005F90> 0
2 <PIL.Image.Image image mode=RGB size=128x64 at 0x7F89B50165D0> 0
3 <PIL.Image.Image image mode=RGB size=128x64 at 0x7F89B5016110> 0
4 <PIL.Image.Image image mode=RGB size=128x64 at 0x7F89B51477D0> 0
5 <PIL.Image.Image image mode=RGB size=128x64 at 0x7F89B5005B90> 0
6 <PIL.Image.Image image mode=RGB size=128x64 at 0x7F89B5016110> 0
7 <PIL.Image.Image image mode=RGB size=128x64 at 0x7F89B5005A50> 0
8 <PIL.Image.Image image mode=RGB size=128x64 at 0x7F89B5016850> 0
9 <PIL.Image.Image image mode=RGB size=128x64 at 0x7F89B5016E90> 0
10 <PIL.Image.Image image mode=RGB size=128x64 at 0x7F89B50165D0> 0
11 <PIL.Image.Image image mode=RGB size=128x64 at 0x7F89B5016E90> 0
12 <PIL.Image.Image image mode=RGB size=128x64 at 0x7F89B5016110> 0
13 <PIL.Image.Image image mode=RGB size=128x64 at 0x7F89B50165D0> 0
14 <PIL.Image.Image image mode=RGB size=128x64 at 0x7F89B5

In [25]:
trans = transforms.Compose([
    transforms.ToTensor()
])

train_data = torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/Colab Notebooks/custom_data/train_data', transform=trans)

In [26]:
data_loader = DataLoader(dataset = train_data, batch_size = 8, shuffle = True, num_workers=2)

In [27]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,6,5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6,16,5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer3 = nn.Sequential(
            nn.Linear(16*13*29, 120),
            nn.ReLU(),
            nn.Linear(120,2)
        )
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.shape[0], -1)
        out = self.layer3(out)
        return out

In [28]:
net = CNN().to(device)
test_input = (torch.Tensor(3,3,64,128)).to(device)
test_out = net(test_input)

In [29]:
optimizer = optim.Adam(net.parameters(), lr=0.00005)
loss_func = nn.CrossEntropyLoss().to(device)

In [30]:
total_batch = len(data_loader)

epochs = 7
for epoch in range(epochs):
    avg_cost = 0.0
    for num, data in enumerate(data_loader):
        imgs, labels = data
        imgs = imgs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        out = net(imgs)
        loss = loss_func(out, labels)
        loss.backward()
        optimizer.step()
        
        avg_cost += loss / total_batch
        
    print('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))

[Epoch:1] cost = 0.5987470746040344
[Epoch:2] cost = 0.295551598072052
[Epoch:3] cost = 0.08803733438253403
[Epoch:4] cost = 0.029591666534543037
[Epoch:5] cost = 0.014310908503830433
[Epoch:6] cost = 0.0077603585086762905
[Epoch:7] cost = 0.004863244481384754


In [33]:
torch.save(net.state_dict(), "/content/drive/MyDrive/Colab Notebooks/model/model.pth")

In [34]:
new_net = CNN().to(device)

In [35]:
new_net.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks//model/model.pth'))

<All keys matched successfully>

In [36]:
print(net.layer1[0])
print(new_net.layer1[0])

print(net.layer1[0].weight[0][0][0])
print(new_net.layer1[0].weight[0][0][0])

net.layer1[0].weight[0] == new_net.layer1[0].weight[0]

Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
tensor([-0.0235,  0.0602,  0.0666,  0.0730, -0.0979], device='cuda:0',
       grad_fn=<SelectBackward0>)
tensor([-0.0235,  0.0602,  0.0666,  0.0730, -0.0979], device='cuda:0',
       grad_fn=<SelectBackward0>)


tensor([[[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]],

        [[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]],

        [[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]]], device='cuda:0')

In [37]:
trans=torchvision.transforms.Compose([
    transforms.Resize((64,128)),
    transforms.ToTensor()
])
test_data = torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/Colab Notebooks/custom_data/test_data', transform=trans)

In [38]:
test_set = DataLoader(dataset = test_data, batch_size = len(test_data))

In [39]:
with torch.no_grad():
    for num, data in enumerate(test_set):
        imgs, label = data
        imgs = imgs.to(device)
        label = label.to(device)
        
        prediction = net(imgs)
        
        correct_prediction = torch.argmax(prediction, 1) == label
        
        accuracy = correct_prediction.float().mean()
        print('Accuracy:', accuracy.item())

Accuracy: 1.0
